In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.integrate import quad

**Exercise 14.1**

In [2]:
def integrate(g, a, b, N, method):
    
    i = np.arange(0, N, 1)
    x = a + ((2 * i + 1) * (b - a)) / (2 * N)
    
    if method == 'midpoint':
        return ((b - a) / N) * np.sum(g(x))
    
    if method == 'trapezoid':
        return (b - a) / (2 * N) * (g(x[0]) + 2 * np.sum(g(x[1:-1])) + g(x[-1]))
    
    if method == 'simpson':
        i = np.arange(0, 2 * N, 1)
        x = a + (i * (b - a)) / (2 * N)
        return ((b - a) / (6 * (N + 1))) * (g(x[0]) + 4 * np.sum(g(x[1:(2*N-1):2])) + 2 \
            * np.sum(g(x[2:(2*N-2):2])) + g(x[-1]))

In [3]:
g = lambda x: 0.1 * x ** 4 - 1.5 * x ** 3 + 0.53 * x ** 2 + 2 * x + 1

In [4]:
integrate(g, -10, 10, 500, 'midpoint')

4373.278586816

In [5]:
integrate(g, -10, 10, 1000, 'trapezoid')

4352.321645695981

In [6]:
integrate(g, -10, 10, 500, 'simpson')

4381.574824044178

**Exercise 14.2**

In [7]:
def distr_approx(μ, σ, N, k):
    dist = norm(loc=μ, scale=σ)
    ω = np.zeros(N)
    z = np.linspace(μ - k * σ, μ + k * σ, N)
    ω[0] = dist.cdf((z[0] + z[1]) / 2)
    
    for i in range(1, N-1):
        z_min = (z[i-1] + z[i]) / 2
        z_max = (z[i] + z[i+1]) / 2
        ω[i] = dist.cdf(z_max) - dist.cdf(z_min)
    
    ω[-1] = 1 - dist.cdf(z[-2] + z[-1])
    return z, ω

In [8]:
nodes, weights = distr_approx(0, 1, 11, 3)
nodes

array([-3. , -2.4, -1.8, -1.2, -0.6,  0. ,  0.6,  1.2,  1.8,  2.4,  3. ])

In [9]:
weights

array([3.46697380e-03, 1.43974468e-02, 4.89427807e-02, 1.17252924e-01,
       1.98028452e-01, 2.35822844e-01, 1.98028452e-01, 1.17252924e-01,
       4.89427807e-02, 1.43974468e-02, 3.33204485e-08])

**Exercise 14.3**

In [10]:
def logdistr_approx(μ, σ, N, k):
    z, ω = distr_approx(μ, σ, N, k)
    return np.exp(z), ω

In [11]:
nodes, weights = logdistr_approx(0, 1, 11, 3)
nodes

array([ 0.04978707,  0.09071795,  0.16529889,  0.30119421,  0.54881164,
        1.        ,  1.8221188 ,  3.32011692,  6.04964746, 11.02317638,
       20.08553692])

In [12]:
weights

array([3.46697380e-03, 1.43974468e-02, 4.89427807e-02, 1.17252924e-01,
       1.98028452e-01, 2.35822844e-01, 1.98028452e-01, 1.17252924e-01,
       4.89427807e-02, 1.43974468e-02, 3.33204485e-08])

**Exercise 14.4**

In [13]:
μ = 10.5
σ = 0.8
nodes, weights = logdistr_approx(μ, σ, 11, 3)

In [14]:
np.sum(nodes * weights)

48964.58431238742

In [15]:
np.exp(μ + σ ** 2 / 2)

50011.087008521754

**Exercise 14.5**

In [16]:
g = lambda x: 0.1 * x ** 4 - 1.5 * x ** 3 + 0.53 * x ** 2 + 2 * x + 1

In [17]:
def quadrature(params, a, b):
    
    ω, x = params[:3], params[3:]
    eqs = np.empty(len(params))
    
    for i in range(1, len(params)+1):
        eqs[i-1] = ((b ** i - a ** i) / i) - ω @ x ** (i - 1)
        
    return eqs

In [18]:
params = root(quadrature, x0=np.ones(6), args=(-10, 10)).x
weights, nodes = params[:3], params[3:]
weights, nodes
np.sum(weights * g(nodes))

4373.333333340382

**Exercise 14.6**

In [19]:
quad(g, -10, 10)

(4373.333333333334, 8.109531705284936e-11)

**Exercise 14.7**

In [20]:
def approx(g, min_x, max_x, N):
    rand_draws = np.random.uniform(min_x, max_x, size=(N, 2))
    return 4 * (1/N) * np.sum(g(rand_draws[:, 0], rand_draws[:, 1]))

In [21]:
def g(x, y):
    return x ** 2 + y ** 2 <= 1

In [22]:
approx(g, -1, 1, 10000)

3.16

In [23]:
toler = 1e-5
diff = 1e3
n = 10
max_N = 100_000
while diff > toler and n < max_N:
    π = approx(g, -1, 1, n)
    diff = np.abs(π - 3.1415)
    n += 1

In [24]:
n, π

(1478, 3.1415030467163167)

**Exercise 14.8**

In [25]:
def n_p(n):
    
    primes = [2]
    num = 3
    while len(primes) < n:
        for p in primes:
            if num % p == 0:
                break
        else:
            primes.append(num)
        num += 2
    return primes

In [26]:
def equidistr(n, d, seq):
    
    if seq == 'weyl':
        prime = n_p(d)
        x = n * np.power(prime, .5)
        xn = x - np.floor(x)
        
    elif seq == 'haber':
        prime = n_p(d)
        x = (n * (n + 1.0)) / 2 * np.power(prime, .5)
        xn = x - np.floor(x)
        
    elif seq == 'niederreiter':
        ar = np.arange(1, d + 1)
        x = n * (2 ** (ar / (d + 1)))
        x_floor = np.floor(x)
        xn = x - x_floor
        
    elif seq == 'baker':
        prime = n_p(d)
        x = n * np.exp(prime)
        xn = x - np.floor(x)
    else:
        raise ValueError('Sequence does not exist.')
        
    return xn

In [27]:
print("weyl:", equidistr(10, 2, 'weyl'))
print("haber:", equidistr(10, 2, 'haber'))
print("niederreiter", equidistr(10, 2, 'niederreiter'))
print("baker", equidistr(5, 2, 'baker'))

weyl: [0.14213562 0.32050808]
haber: [0.78174593 0.26279442]
niederreiter [0.5992105  0.87401052]
baker [0.94528049 0.42768462]


**Exercise 14.9**

In [28]:
def approx_149(N, seq):
    
    π = 0
    iterate = 0
    s = np.zeros((N, 2), dtype=np.float64)
    for i in range(1, N+1):
        s[i-1, :] = equidistr(i, 2, seq)
    
    for j in range(N):
        if s[j,0] ** 2 + s[j,1] ** 2 <= 1:
            iterate += 1
    π = 4.0 * iterate / N
    
    return π

In [29]:
print("weyl:", approx_149(100000, 'weyl'))
print("haber:", approx_149(100000, 'haber'))
print("niederreiter", approx_149(100000, 'niederreiter'))
print("baker", approx_149(100000, 'baker'))

weyl: 3.14036
haber: 3.14396
niederreiter 3.1414
baker 3.1416
